In [5]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import year, month
from pyspark.sql.functions import to_timestamp,date_format

spark = SparkSession.builder.getOrCreate()

In [6]:
df1 = spark.read.load('/mnt/prepared/SupplyChain/StoreOrder/StoreOrderLine')
df2 = spark.read.load('dbfs:/mnt/thorcf2/clean/supp/dedi/storeorder')

In [7]:
df2 = (
    df2.select("Header.*", "Payload.*", "DeliveryDatePartitionKey", "__meta.*")
    .select("Event.*", "StoreOrder.*", "DeliveryDatePartitionKey", "ModifiedTimestamp")
    .select("Action", "DateTime", "Opco.*", "StoreNo", "WarehouseNo","StoreOrderNo","MessageCreationDatetime","DeliveryStreamNo","DeliveryDateTime",
            "DelivererNo","SupplyMethod","StoreOrderLine","DeliveryDatePartitionKey","ModifiedTimestamp")
    .select("Action","DateTime","CountryCode","GLN","StoreNo","WarehouseNo","StoreOrderNo","MessageCreationDatetime",
            "DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod",*[F.col('StoreOrderLine')[i].alias(f'StoreOrderLine{i}') for i in range(1)],
            "DeliveryDatePartitionKey","ModifiedTimestamp")
    .select("Action","DateTime","CountryCode","GLN","StoreNo","WarehouseNo","StoreOrderNo","MessageCreationDatetime","DeliveryStreamNo","DeliveryDateTime","DelivererNo","SupplyMethod","StoreOrderLine0.*","DeliveryDatePartitionKey","ModifiedTimestamp")
)
df2 = df2.withColumnRenamed('SoiNo','Soino')
df2 = df2.withColumn("date", to_date(col("DeliveryDateTime")))
df2 = df2.withColumn('year', year(df2.date))
df2 = df2.withColumn('month', month(df2.date))
df2 = df2.withColumn("date",to_timestamp(col("date"))).withColumn("day", date_format(col("date"), "d"))

df2.printSchema()

root
 |-- Action: string (nullable = true)
 |-- DateTime: timestamp (nullable = true)
 |-- CountryCode: string (nullable = true)
 |-- GLN: long (nullable = true)
 |-- StoreNo: integer (nullable = true)
 |-- WarehouseNo: integer (nullable = true)
 |-- StoreOrderNo: integer (nullable = true)
 |-- MessageCreationDatetime: timestamp (nullable = true)
 |-- DeliveryStreamNo: integer (nullable = true)
 |-- DeliveryDateTime: timestamp (nullable = true)
 |-- DelivererNo: integer (nullable = true)
 |-- SupplyMethod: string (nullable = true)
 |-- Soino: long (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- OrderLineId: integer (nullable = true)
 |-- PromotionInd: string (nullable = true)
 |-- InitialPromotionInd: string (nullable = true)
 |-- SupplierCompanyGln: long (nullable = true)
 |-- SupplierNo: integer (nullable = true)
 |-- AddUnorderedInd: string (nullable = true)
 |-- EmergencyOrderInd: string (nullable = true)
 |-- DeliveryDatePartitionKey: integer (nullable = true)
 |--